In [ ]:
! pip install sequtils
! pip install Bio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 283 kB 6.8 MB/s 
     |████████████████████████████████| 147 kB 34.8 MB/s 
     |████████████████████████████████| 178 kB 47.1 MB/s 
     |████████████████████████████████| 40 kB 2.2 MB/s 
     |████████████████████████████████| 1.0 MB 53.5 MB/s 
     |████████████████████████████████| 67 kB 2.4 MB/s 
     |████████████████████████████████| 61 kB 4.1 MB/s 
     |████████████████████████████████| 54 kB 1.2 MB/s 
     |████████████████████████████████| 75 kB 1.7 MB/s 
     |████████████████████████████████| 153 kB 46.3 MB/s 
     |████████████████████████████████| 48 kB 3.3 MB/s 
     |████████████████████████████████| 4.0 MB 43.9 MB/s 
  ERROR: Failed building wheel for sequtils
  Running setup.py clean for sequtils
  Created wheel for environ: filename=environ-1.0-py3-none-any.whl size=3246 sha256=aa023088e27d19756be7bc4f606db230d28d282508baa2e5757b

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

In [3]:
# Will take 3-5 seconds to run
def load_fixed_train_df(original_train_file_path="/content/drive/Shareddrives/2:1 Caitlin & Kimai/Data/Enzyme Stability Prediction/train.csv",
                        update_file_path="/content/drive/Shareddrives/2:1 Caitlin & Kimai/Data/Enzyme Stability Prediction/train_updates_20220929.csv",
                        was_fixed_col=False):
    def _fix_tm_ph(_row, update_map):
        update_vals = update_map.get(_row["seq_id"], None)
        if update_vals is not None:
            _row["tm"] = update_vals["tm"]
            _row["pH"] = update_vals["pH"]
        return _row

    # Load dataframes
    _df = pd.read_csv(original_train_file_path)
    _updates_df = pd.read_csv(update_file_path)

    # Identify which sequence ids need to have the tm and pH values changed and create a dictionary mapping 
    seqid_2_phtm_update_map = _updates_df[~pd.isna(_updates_df["pH"])].groupby("seq_id")[["pH", "tm"]].first().to_dict("index")

    # Identify the sequence ids that will be dropped due to data quality issues
    bad_seqids = _updates_df[pd.isna(_updates_df["pH"])]["seq_id"].to_list()

    # Fix bad sequence ids
    _df = _df[~_df["seq_id"].isin(bad_seqids)].reset_index(drop=True)

    # Fix pH and tm swaparoo
    _df = _df.apply(lambda x: _fix_tm_ph(x, seqid_2_phtm_update_map), axis=1)

    # Add in a bool to track if a row was fixed or not (tm/ph swap will look the same as bad data)
    if was_fixed_col: _df["was_fixed"] = _df["seq_id"].isin(bad_seqids+list(seqid_2_phtm_update_map.keys()))

    return _df

In [4]:
data = load_fixed_train_df()
data.head()
data.to_csv("train_ps.csv", index = False)

In [ ]:
data.isnull().sum()

seq_id                0
protein_sequence      0
pH                  286
data_source         980
tm                    0
dtype: int64

In [ ]:
data['pH'].fillna(data['pH'].mean(), inplace = True)

In [ ]:
data['length'] = data['protein_sequence'].str.len()

In [ ]:
def protein_analyser(row):
  from Bio.SeqUtils.ProtParam import ProteinAnalysis
  #selecting protein sequences
  sequence = row['protein_sequence']
  analyse_seq = ProteinAnalysis(sequence)

  #count amino acids
  row['amino_acid_count'] = analyse_seq.count_amino_acids()

  return row

data = data.apply(lambda row: protein_analyser(row), axis = 1)
data.head()

,seq_id,protein_sequence,pH,data_source,tm,length,amino_acid_count
0,0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,doi.org/10.1038/s41592-020-0801-4,75.7,341,"{'A': 45, 'C': 1, 'D': 13, 'E': 30, 'F': 13, '..."
1,1,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,doi.org/10.1038/s41592-020-0801-4,50.5,286,"{'A': 28, 'C': 0, 'D': 10, 'E': 52, 'F': 6, 'G..."
2,2,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,doi.org/10.1038/s41592-020-0801-4,40.5,497,"{'A': 50, 'C': 9, 'D': 27, 'E': 32, 'F': 21, '..."
3,3,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,7.0,doi.org/10.1038/s41592-020-0801-4,47.2,265,"{'A': 20, 'C': 5, 'D': 19, 'E': 29, 'F': 12, '..."
4,4,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,7.0,doi.org/10.1038/s41592-020-0801-4,49.5,1451,"{'A': 86, 'C': 14, 'D': 78, 'E': 78, 'F': 32, ..."


In [ ]:
#creating seperate columns for seperate acids
def amino_acid_count(row):

  row1 = row['amino_acid_count']
  for key,value in row1.items():
    row[key] = value

  return row

#setting the columns
data = data.apply(lambda x: amino_acid_count(x), axis = 1)
data.head()

,seq_id,protein_sequence,pH,data_source,tm,length,amino_acid_count,A,C,D,...,M,N,P,Q,R,S,T,V,W,Y
0,0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,doi.org/10.1038/s41592-020-0801-4,75.7,341,"{'A': 45, 'C': 1, 'D': 13, 'E': 30, 'F': 13, '...",45,1,13,...,8,5,18,6,25,11,14,37,4,3
1,1,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,doi.org/10.1038/s41592-020-0801-4,50.5,286,"{'A': 28, 'C': 0, 'D': 10, 'E': 52, 'F': 6, 'G...",28,0,10,...,2,6,8,22,30,14,12,13,3,3
2,2,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,doi.org/10.1038/s41592-020-0801-4,40.5,497,"{'A': 50, 'C': 9, 'D': 27, 'E': 32, 'F': 21, '...",50,9,27,...,6,15,20,25,31,33,30,30,3,16
3,3,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,7.0,doi.org/10.1038/s41592-020-0801-4,47.2,265,"{'A': 20, 'C': 5, 'D': 19, 'E': 29, 'F': 12, '...",20,5,19,...,2,9,16,9,10,16,19,14,3,4
4,4,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,7.0,doi.org/10.1038/s41592-020-0801-4,49.5,1451,"{'A': 86, 'C': 14, 'D': 78, 'E': 78, 'F': 32, ...",86,14,78,...,31,65,128,54,63,148,120,124,16,47


In [ ]:
data.to_csv("ps_with_amino_acid_count.csv", index = False)

In [ ]:
data = pd.read_csv("/content/ps_with_amino_acid_count.csv")
data.head()

,seq_id,protein_sequence,pH,data_source,tm,length,amino_acid_count,A,C,D,...,M,N,P,Q,R,S,T,V,W,Y
0,0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,doi.org/10.1038/s41592-020-0801-4,75.7,341,"{'A': 45, 'C': 1, 'D': 13, 'E': 30, 'F': 13, '...",45,1,13,...,8,5,18,6,25,11,14,37,4,3
1,1,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,doi.org/10.1038/s41592-020-0801-4,50.5,286,"{'A': 28, 'C': 0, 'D': 10, 'E': 52, 'F': 6, 'G...",28,0,10,...,2,6,8,22,30,14,12,13,3,3
2,2,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,doi.org/10.1038/s41592-020-0801-4,40.5,497,"{'A': 50, 'C': 9, 'D': 27, 'E': 32, 'F': 21, '...",50,9,27,...,6,15,20,25,31,33,30,30,3,16
3,3,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,7.0,doi.org/10.1038/s41592-020-0801-4,47.2,265,"{'A': 20, 'C': 5, 'D': 19, 'E': 29, 'F': 12, '...",20,5,19,...,2,9,16,9,10,16,19,14,3,4
4,4,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,7.0,doi.org/10.1038/s41592-020-0801-4,49.5,1451,"{'A': 86, 'C': 14, 'D': 78, 'E': 78, 'F': 32, ...",86,14,78,...,31,65,128,54,63,148,120,124,16,47


In [ ]:
data.drop(["seq_id","protein_sequence","data_source","amino_acid_count"], axis = 1, inplace = True)
data.head()

,pH,tm,length,A,C,D,E,F,G,H,...,M,N,P,Q,R,S,T,V,W,Y
0,7.0,75.7,341,45,1,13,30,13,38,3,...,8,5,18,6,25,11,14,37,4,3
1,7.0,50.5,286,28,0,10,52,6,18,4,...,2,6,8,22,30,14,12,13,3,3
2,7.0,40.5,497,50,9,27,32,21,65,11,...,6,15,20,25,31,33,30,30,3,16
3,7.0,47.2,265,20,5,19,29,12,16,7,...,2,9,16,9,10,16,19,14,3,4
4,7.0,49.5,1451,86,14,78,78,32,84,40,...,31,65,128,54,63,148,120,124,16,47


In [ ]:
#modeling
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np

X = data.drop('tm', axis = 1)
Y = data['tm']

x_train, x_test, y_train,y_test = train_test_split(X, Y, test_size = 0.2)

#model fitting
model = RandomForestRegressor()
model.fit(x_train, y_train)

#predictions
y_pred = model.predict(x_test)
print("R2_score is : {}".format(r2_score(y_test,y_pred)))
print("mean absolute error Test: {}".format(mean_absolute_error(y_pred,y_test)))
print("mean absolute error Train: {}".format(mean_absolute_error(y_train, model.predict(x_train))))


R2_score is : 0.5903773967289432
mean absolute error Test: 5.611702426046168
mean absolute error Train: 2.206785761256686


In [ ]:
# #modeling
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.neighbors import KNeighborsRegressor
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
# import numpy as np

# X = data.drop('tm', axis = 1)
# Y = data['tm']

# x_train, x_test, y_train,y_test = train_test_split(X, Y, test_size = 0.33, random_state=42)

# #model fitting
# model = KNeighborsRegressor(n_neighbors = 5)
# model.fit(x_train, y_train)

# #predictions
# y_pred = model.predict(x_test)
# print("R2_score is : {}".format(r2_score(y_test,y_pred)))
# print("mean absolute error Test: {}".format(mean_absolute_error(y_pred,y_test)))
# print("mean absolute error Train: {}".format(mean_absolute_error(y_train, model.predict(x_train))))

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
from scipy.stats import randint

random_grid={'max_depth':list(np.arange(10, 100, step=10)),
              'n_estimators':np.arange(10, 500, step=50),
              'max_features':randint(1,7),
              'min_samples_leaf':randint(1,4),
              'min_samples_split':np.arange(2, 10, step=2)
         }

rf = RandomForestRegressor(random_state = 42)
rf_random = RandomizedSearchCV(estimator=rf, param_distributions=random_grid,
                              n_iter = 50, scoring='neg_mean_absolute_error', 
                              cv =3, verbose=2, random_state=42, n_jobs=-1,
                              return_train_score=True)

# Fit the random search model
rf_random.fit(x_train,y_train)

In [ ]:
rf_random.best_estimator_

RandomForestRegressor(max_depth=30, max_features=1, n_estimators=360,
                      random_state=42)

## Submission Creation

In [ ]:
test_data = pd.read_csv("/content/drive/Shareddrives/2:1 Caitlin & Kimai/Data/Enzyme Stability Prediction/test.csv")
test_data['length'] = test_data['protein_sequence'].str.len()
test_data = test_data.apply(lambda row: protein_analyser(row), axis = 1)
test_data = test_data.apply(lambda x: amino_acid_count(x), axis = 1)
test_data.drop(["seq_id","protein_sequence","data_source","amino_acid_count"], axis = 1, inplace = True)
test_data.head()

,pH,length,A,C,D,E,F,G,H,I,...,M,N,P,Q,R,S,T,V,W,Y
0,8,221,22,4,15,8,10,19,0,6,...,0,19,17,13,3,18,8,13,6,6
1,8,221,22,4,15,7,10,19,0,6,...,0,19,17,13,3,18,8,13,6,6
2,8,220,22,4,15,7,10,19,0,6,...,0,19,17,13,3,18,8,13,6,6
3,8,221,22,5,15,7,10,19,0,6,...,0,19,17,13,3,18,8,13,6,6
4,8,221,22,4,15,7,11,19,0,6,...,0,19,17,13,3,18,8,13,6,6


In [ ]:
sample_data = pd.read_csv("/content/drive/Shareddrives/2:1 Caitlin & Kimai/Data/Enzyme Stability Prediction/sample_submission.csv")
sample_data['tm'] = model.predict(test_data)
sample_data.head()

,seq_id,tm
0,31390,68.430377
1,31391,68.430377
2,31392,68.430377
3,31393,68.605877
4,31394,68.462377


In [ ]:
sample_data.to_csv("sub1_RF_last.csv", index = False)